# Classification of Parkinsson patients from EEG signals #
Ekaterina Lavrova 

## Import Packages ##

In [ ]:
#import packages 
import os
import numpy as np
import pandas as pd 
import mne 
from mne.preprocessing import ICA
from scipy.signal import welch 
from sklearn.metrics import accuracy_score,recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler  

 ## Import Data ##

In [ ]:
#import data
file_path = "C:/Users/katja/Programing_project/Data" 
raw_list = []

for file in os.listdir(file_path):
    raw = mne.io.read_raw_bdf(os.path.join(file_path,file), preload = True,verbose = False) 
    raw_list.append(raw)

## Filtering ##

In [ ]:
#Filtering 
raw_test = raw_list[1]
raw_test.plot(start = 0, duration = 30,scalings = dict(eeg=200e-6)) # Sclaed plot of file with index 1 

for raw in raw_list: 
    raw.filter(1.0,40.0, verbose = False) # Bandpass filtering 
    raw.notch_filter(50.0, verbose = False) # Notch filtering ( 50 Hz european standards, change to 60 Hz in USA)
    raw.set_eeg_reference('average', verbose = False) #Rereferencing using average method

raw_test.plot(start = 0, duration = 30,scalings = dict(eeg=200e-6)) #Plot after filtering

## Prepare ICA and data for artifact removal ##

In [ ]:
ica_list = [] #Empty list for ICA components 

for raw in raw_list:
    montage = mne.channels.make_standard_montage('standard_1020') # Give channels positions according to the 10 20 system 

    # Mark EXG channels as NA 
    raw.set_channel_types({
        'EXG1':'misc', 'EXG2':'misc', 'EXG3':'misc', 'EXG4':'misc',
        'EXG5':'misc', 'EXG6':'misc', 'EXG7':'misc', 'EXG8':'misc'
    })
    raw.set_montage(montage)

# Configure and fit ICA to the data
for raw in raw_list:
    ica = ICA(n_components=15, method='fastica', random_state=42, verbose = False)
    ica.fit(raw, verbose = False )
    ica_list.append(ica)

## Plot ICA components to identify artifacts ##

In [ ]:
# Creates variable  that only include one file
raw_test = raw_list[1]
ica_test = ica_list[1]

# Plot components of one file 
ica_test.plot_components()

# Plot overlay of raw signal with specific ICA components excluded for one file 
ica_test.plot_overlay(raw_test, exclude=[0])  # exclude component 0 in file idx 1
ica_test.plot_overlay(raw_test, exclude=[1])  # exclude component 1 in file idx 1

## Apply ICA filtering to remove artifacts ## 

In [ ]:
#Apply ICA filtering 
processed_raw = []

#Combine ICA and and the raw datafiles 
for ica, raw in zip(ica_list, raw_list):
    ica.exclude = [0,1] #exclude components 

    raw_clean = ica.apply(raw.copy(), verbose = False) #Create copies of each raw file xith ica applied 
    processed_raw.append(raw_clean) # add raw_copies to list

# Plot one fiel from the processed list to compare with unfiltered plot
raw = processed_raw [1]
raw.plot (start = 0, duration = 30, scalings=dict(eeg=200e-6))